In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
file_path = '/Users/simgyuseong/Documents/Hate_Detection/dataset/raw/KHS_dataset/KHS_dataset.txt'
data = pd.read_csv(file_path, delimiter=',')

# Display the first few rows of the dataset
print(data.head())

# Define the labels
labels = ['contain_gender' , 'bias' , 'hate']
# Text cleaning function
def clean_text(text):
    # Remove non-alphanumeric characters and extra spaces
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"[^a-zA-Z가-힣0-9\s]", "", text)
    return text.strip()

# Clean the 'comments' column
data['comments'] = data['comments'].apply(clean_text)

# Convert labels to a list of labels
def extract_labels(row):
    return [labels[i] for i in range(len(labels)) if row[i+1] == 1]

data['labels'] = data.apply(extract_labels, axis=1)

# Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['comments'])
sequences = tokenizer.texts_to_sequences(data['comments'])

# Pad the sequences
maxlen = 100  # You can adjust the maxlen according to your needs
X = pad_sequences(sequences, maxlen=maxlen)

# Binarize the labels
mlb = MultiLabelBinarizer(classes=labels)
y = mlb.fit_transform(data['labels'])

# Print the shape of the dataset
print(f'Shape of X: {X.shape}')
print(f'Shape of y: {y.shape}')




                                            comments  contain_gender_bias  \
0  (현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...                    0   
1  ....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...                    0   
2  ...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...                    0   
3                 1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데                    0   
4  1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...                    1   

   bias  hate  
0     1     1  
1     0     0  
2     0     1  
3     0     0  
4     1     1  


/var/folders/sd/pq30895j7xl996rt4n_pcm0r0000gn/T/ipykernel_77541/4099103749.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return [labels[i] for i in range(len(labels)) if row[i+1] == 1]


Shape of X: (7896, 100)
Shape of y: (7896, 3)


In [34]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 37086
등장 빈도가 2번 이하인 희귀 단어의 수: 33817
단어 집합에서 희귀 단어의 비율: 91.1853529633824
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 57.118334776691825


In [35]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 3271
